# COMP90051 Project 2

In [23]:
# Do not edit. These are the only imports permitted.
%matplotlib inline
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

## 1. Implementing ε-Greedy and UCB

In [24]:
class MAB(ABC):
    """
    Abstract class that represents a multi-armed bandit (MAB)
    """
    
    @abstractmethod
    def play(self, tround, context):
        """
        Play a round
        
        Arguments
        =========
        tround : int
            positive integer identifying the round
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to the arms
        
        Returns
        =======
        arm : int
            the positive integer arm id for this round
        """
    
    @abstractmethod
    def update(self, arm, reward, context):
        """
        Updates the internal state of the MAB after a play
        
        Arguments
        =========
        arm : int
            a positive integer arm id in {1, ..., self.narms}
        
        reward : float
            reward received from arm
        
        context : 1D float array, shape (self.ndims * self.narms), optional
            context given to arms
        """

In [64]:
class EpsGreedy(MAB):
    """
    Epsilon-Greedy multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    epsilon : float
        explore probability

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, epsilon, Q0=np.inf):
        self.epsilon = epsilon
        self.narms = narms
        self.estimated_rewards = np.full(self.narms,Q0)
        self.number_of_rounds = np.zeros(self.narms)
        
        
    def play(self, tround, context=None):
        ram = np.random.random()
        if  ram < self.epsilon:
            index = np.random.randint(0, self.narms)
        else:
            index = np.argmax(self.estimated_rewards)
        arm = index + 1
        return arm
        
    def update(self, arm, reward, context=None):
        index = arm -1
        cv = self.estimated_rewards[index] # current value of index arm
        cr = self.number_of_rounds[index] # current round of index arm
        self.estimated_rewards[index] = (cr/(cr+1))*cv + (1/(cr+1))*reward if cr > 0 else reward # update with reward
        self.number_of_rounds[index] += 1 # once updated, round +1
    

In [196]:
class UCB(MAB):
    """
    Upper Confidence Bound (UCB) multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    rho : float
        positive real explore-exploit parameter

    Q0 : float, optional
        initial value for the arms
    """
    
    def __init__(self, narms, rho, Q0=np.inf):
        self.rho = rho
        self.narms = narms
        self.estimated_rewards = np.full(self.narms, Q0)
        self.chosen_count = np.zeros(self.narms)
        self.total_rewards = np.zeros(self.narms)
    
    def play(self, tround, context=None):
        def calculate_delta(round_number, item):
            return np.sqrt(self.rho * np.log(round_number) / self.chosen_count[item])
        
        zero_indices = np.where(self.chosen_count == 0)[0]
        if len(zero_indices) > 0:
            index = np.random.choice(zero_indices)
        else:
            upper_bound_probs = [self.total_rewards[item]/self.chosen_count[item] + calculate_delta(tround, item) for item in range(self.narms)]
            index = np.argmax(upper_bound_probs)
        arm = index + 1
        return arm
#         arms = np.argwhere(self.estimated_rewards == np.amax(self.estimated_rewards)).flatten().tolist()
#         return ramdon.choice(arms)+1
        
    def update(self, arm, reward, context=None):
        index = arm -1
        c_value = self.estimated_rewards[index] # current value of index arm
        c_round = self.chosen_count[index] # current round of index arm
        self.estimated_rewards[index] = (c_round/(c_round+1))*c_value + (1/(c_round+1))*reward # update with reward
        self.total_rewards[index] += reward
        self.chosen_count[index] += 1 # once updated, round +1
        

## 2. The Basic Thompson Bandit

In [197]:
class BetaThompson(MAB):
    """
    Beta-Bernoulli Thompson sampling multi-armed bandit

    Arguments
    =========
    narms : int
        number of arms

    alpha0 : float, optional
        positive real prior hyperparameter

    beta0 : float, optional
        positive real prior hyperparameter
    """
    
    def __init__(self, narms, alpha0=1.0, beta0=1.0):
        self.alphas = [alpha0] * narms
        self.betas = [beta0] * narms
        self.narms = narms
        self.succeses = np.zeros(self.narms)
        self.failures = np.zeros(self.narms)
        
    
    def play(self, tround, context=None):
        import random
        thetas = [random.betavariate(self.succeses[i] + self.alphas[i],
            self.failures[i] + self.betas[i]) for i in range(self.narms)]
        index = np.argmax(thetas)
        return index + 1
        
    def update(self, arm, reward, context=None):
        if reward == 1:
            self.succeses[arm-1] += 1
        else:
            self.failures[arm-1] += 1

## 3. Off-Policy Evaluation

In [198]:
def offlineEvaluate(mab, arms, rewards, contexts, nrounds=None):
    """
    Offline evaluation of a multi-armed bandit
    
    Arguments
    =========
    mab : instance of MAB
    
    arms : 1D int array, shape (nevents,) 
        integer arm id for each event
    
    rewards : 1D float array, shape (nevents,)
        reward received for each event
    
    contexts : 2D float array, shape (nevents, mab.narms*nfeatures)
        contexts presented to the arms (stacked horizontally) 
        for each event.
        
    nrounds : int, optional
        number of matching events to evaluate `mab` on.
    
    Returns
    =======
    out : 1D float array
        rewards for the matching events
    """
    
    # history = []
    rewards_list = []

    # exit when 0 rounds
    if nrounds == 0:
        return 0

    tround = 1
    idx = 0
    while tround <= nrounds:
        if idx >= len(contexts) - 1:
            break

        # get next event
        context = contexts[idx]
        arm = arms[idx]
        reward = rewards[idx]
        idx += 1

        chosen_arm = mab.play(tround, context)
        # when arm matching
        if chosen_arm == arm:
            mab.update(arm, reward, context)
            # history.append((context, arm, reward))
            rewards_list.append(reward)
            tround += 1

    return rewards_list

In [199]:
data = np.genfromtxt("dataset.txt", delimiter=" ")
arms = data[:, 0].astype(np.int64)
rewards = data[:, 1]
contexts = data[:, 2:]


In [200]:
mab = EpsGreedy(10, 0.05)
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

EpsGreedy average reward 0.25875


In [201]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('UCB average reward', np.mean(results_UCB))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in double_scalars


UCB average reward 0.17625


In [202]:
mab = BetaThompson(10, 1.0, 1.0)
results_BetaThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('BetaThompson average reward', np.mean(results_BetaThompson))

BetaThompson average reward 0.22375


## 4. Contextual Bandits - LinUCB

In [ ]:
class LinUCB(MAB):
    """
    Contextual multi-armed bandit (LinUCB)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    alpha : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, alpha):
        self.narms = narms
        self.ndims = ndims
        self.alpha = alpha
        self.arm_range = range(1, narms + 1)
        self.Aa_dict = {arm: np.identity(ndims) for arm in self.arm_range}
        self.ba_dict = {arm: np.zeros(ndims) for arm in self.arm_range}
        
        
    def play(self, tround, context):
        
        
    
    def update(self, arm, reward, context):
        
    

In [ ]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinUCB average reward', np.mean(results_LinUCB))

## 5. Contextual Bandits - LinThompson

In [ ]:
class LinThompson(MAB):
    """
    Contextual Thompson sampled multi-armed bandit (LinThompson)

    Arguments
    =========
    narms : int
        number of arms

    ndims : int
        number of dimensions for each arm's context

    v : float
        positive real explore-exploit parameter
    """
    
    def __init__(self, narms, ndims, v):
        
        
    def play(self, tround, context):
        
    
    def update(self, arm, reward, context):
        
    

In [ ]:
mab = LinThompson(10, 10, 1.0)
results_LinThompson = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('LinThompson average reward', np.mean(results_LinThompson))

## 6. Evaluation
### 6.A.

### 6.B.